<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/model_assisted_labeling/mal_basics.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/model_assisted_labeling/mal_basics.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Annotation Imports
* This notebook is a high level introduction demonstrating multiple ways to upload your annotations. This will cover the following:
    * Model-assisted labeling - used to provide pre-annotated data for your labelers. This will enable a reduction in the total amount of time to properly label your assets. Model-assisted labeling does not submit the labels automatically, and will need to be reviewed by a labeler for submission.
    * Label Import - used to provide ground truth labels. These can in turn be used and compared against prediction labels, or used as benchmarks to see how your labelers are doing.


* For complete examples see image_mal.ipynb or ner_mal.ipynb

* For information on what types of annotations are supported per data type, refer to this documentation:
    * https://docs.labelbox.com/docs/model-assisted-labeling#option-1-import-via-python-annotation-types-recommended

* Notes:
    * If you are importing more than 1,000 mask annotations at a time, consider submitting separate jobs, as they can take longer than other annotation types to import.
    * Wait until the import job is complete before opening the Editor to make sure all annotations are imported properly.

# Installs

In [ ]:
!pip install -q 'labelbox[data]'

# Imports

In [ ]:
from labelbox.schema.ontology import OntologyBuilder, Tool
from labelbox.schema.media_type import MediaType
from labelbox import Client, LabelingFrontend, LabelImport, MALPredictionImport
from labelbox.data.annotation_types import (
    Label, ImageData, ObjectAnnotation, Rectangle, Point, Radio,
    ClassificationAnnotation, ClassificationAnswer
)
from labelbox.data.serialization import NDJsonConverter
import uuid
import json

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [ ]:
# Add your api key
API_KEY = None
client = Client(api_key=API_KEY)

INFO:labelbox.client:Initializing Labelbox client at 'https://api.labelbox.com/graphql'


---- 
### Steps
1. Make sure project is setup
2. Collect annotations
3. Upload

### Project setup

We will be creating two projects, one for model-assisted labeling, and one for label imports

In [ ]:
# Only update this if you have an on-prem deployment
ontology_builder = OntologyBuilder(tools=[
    Tool(tool=Tool.Type.BBOX, name="box")
])


mal_project = client.create_project(name="image_mal_project", media_type=MediaType.Image)
li_project = client.create_project(name="image_label_import_project", media_type=MediaType.Image)


dataset = client.create_dataset(name="annotation_import_demo_dataset")
test_img_url = "https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg"
data_row = dataset.create_data_row(row_data=test_img_url)
editor = next(client.get_labeling_frontends(where=LabelingFrontend.name == "Editor"))

mal_project.setup(editor, ontology_builder.asdict())
mal_project.datasets.connect(dataset)

li_project.setup(editor, ontology_builder.asdict())
li_project.datasets.connect(dataset)

#### Create Label using Annotation Type Objects
* It is recommended to use the Python SDK's annotation types. Below is an example of a bounding box, which is the Rectangle annotation type

* A more in depth example can be found here:
    * https://docs.labelbox.com/docs/bounding-box-json

In [ ]:
# Create a label with the related data and annotations
# Then we assign the feature schema id based on an existing project's ontology
# Assignment of the feature schema will be done per each project below
rectangle = Rectangle(start=Point(x=30,y=30), end=Point(x=200,y=200))
rectangle_annotation = ObjectAnnotation(value=rectangle, name="box")
image_data = ImageData(uid=data_row.uid)

label = Label(
    data=image_data,
    annotations = [rectangle_annotation]
)

label

Label(uid=None, data=ImageData(im_bytes=None,file_path=None,url=None,arr=None), annotations=[ObjectAnnotation(name='box', feature_schema_id=None, extra={}, value=Rectangle(extra={}, start=Point(extra={}, x=30.0, y=30.0), end=Point(extra={}, x=200.0, y=200.0)), classifications=[])], extra={})

### Model Assisted Labeling 

To do model-assisted labeling, we need to convert a Label object into an NDJSON. 

This is easily done with using the NDJSONConverter class

We will create a Label called mal_label which has the same original structure as the label above

Notes:
* Each label requires a valid feature schema id or name
* the NDJsonConverter takes in a list of labels

In [ ]:
mal_label = Label(
    data=image_data,
    annotations = [rectangle_annotation]
)

ndjson_labels = list(NDJsonConverter.serialize([mal_label]))

ndjson_labels

In [ ]:
upload_job = MALPredictionImport.create_from_objects(
    client = client, 
    project_id = mal_project.uid, 
    name="upload_label_import_job", 
    predictions=ndjson_labels)

In [ ]:
# Errors will appear for each annotation that failed.
# Empty list means that there were no errors
# This will provide information only after the upload_job is complete, so we do not need to worry about having to rerun
print("Errors:", upload_job.errors)

INFO:labelbox.schema.annotation_import:Sleeping for 10 seconds...


Errors: []


### Label Import

Label import is very similar to model-assisted labeling. We will create a Label called li_label which has the same original structure as the label above

In [ ]:
li_label = Label(
    data=image_data,
    annotations = [rectangle_annotation]
)

ndjson_labels = list(NDJsonConverter.serialize([li_label]))

ndjson_labels

In [ ]:
upload_job = LabelImport.create_from_objects(
    client = client, 
    project_id = li_project.uid, 
    name="upload_label_import_job", 
    labels=ndjson_labels)

In [ ]:
print("Errors:", upload_job.errors)

INFO:labelbox.schema.annotation_import:Sleeping for 10 seconds...


Errors: []
